In [1]:
import pandas as pd

In [2]:
import datacompy

In [3]:
from excel_maker import Support

In [4]:
from postgress_config import PostgressDB

Python-dotenv could not parse statement starting at line 16
Python-dotenv could not parse statement starting at line 23


In [5]:
import time

In [6]:
from oracle_config import Ora

In [7]:
import numpy as np

In [8]:
sup = Support()

In [9]:
main_dataframe = pd.read_excel(r'resources\main_dataframe.xlsx')

In [10]:
main_dataframe

,SERVICE_DATE,PATIENT_ID,EPISODE_ID,PATIENT_NAME,SERVICE_CODE,SERVICE_DESC,PATIENT_TYPE,GROSS_AMOUNT,NET_AMOUNT,DOCTOR_SHARE,DOCTOR_NAME,PHYSICIAN_ID,SOURCE OF DATA,COMMENTS,DOCTORS_GROUP,REFERENCE_GROUP
0,19/06/2023,KH1000854002,14114990,Mrs. Sulochna Rawat,CNIP000025,Doctor Routine Visit - Neurosurgery,Twin,2500.0,2500.0,2500.0,DR. AKSHAT KAYAL,51008200,IP Admitting Doc wise rev rep,NaN,D,IP Consultation
1,01/06/2023,KH1000207883,14132431,Mrs. Sajida Ahmed,CNIP000025,Doctor Routine Visit - Neurosurgery,General,2000.0,2000.0,2000.0,DR. AKSHAT KAYAL,51008200,IP Admitting Doc wise rev rep,NaN,D,IP Consultation
2,02/06/2023,KH1000207883,14132431,Mrs. Sajida Ahmed,CNIP000025,Doctor Routine Visit - Neurosurgery,General,2000.0,2000.0,2000.0,DR. AKSHAT KAYAL,51008200,IP Admitting Doc wise rev rep,NaN,D,IP Consultation
3,03/06/2023,KH1000207883,14132431,Mrs. Sajida Ahmed,CNIP000025,Doctor Routine Visit - Neurosurgery,General,2000.0,2000.0,2000.0,DR. AKSHAT KAYAL,51008200,IP Admitting Doc wise rev rep,NaN,D,IP Consultation
4,04/06/2023,KH1000207883,14132431,Mrs. Sajida Ahmed,CNIP000025,Doctor Routine Visit - Neurosurgery,General,3000.0,3000.0,3000.0,DR. AKSHAT KAYAL,51008200,IP Admitting Doc wise rev rep,NaN,D,IP Consultation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45029,2023-06-29 08:26:24,KH1000885975,143075270001,NaN,CARD,NaN,EHC,400.0,400.0,400.0,Dr.PRAVIN KAHALE,NaN,EHC Query,NaN,C,OP Consultation
45030,2023-06-29 13:12:45,KH1000886167,143086050001,NaN,CARD,NaN,EHC,400.0,400.0,400.0,Dr.PRAVIN KAHALE,NaN,EHC Query,NaN,C,OP Consultation
45031,2023-06-29 13:13:28,KH1000886169,143086080001,NaN,CARD,NaN,EHC,400.0,400.0,400.0,Dr.PRAVIN KAHALE,NaN,EHC Query,NaN,C,OP Consultation
45032,2023-06-29 13:13:54,KH1000886172,143086200001,NaN,CARD,NaN,EHC,400.0,400.0,400.0,Dr.PRAVIN KAHALE,NaN,EHC Query,NaN,C,OP Consultation


In [11]:
#df = pd.read_excel(r'resources\doc.xlsx')
#d = datacompy.Compare(main_dataframe,df,join_columns='PATIENT_NAME')
#sup.excel_generator(excel_data=d.df1_unq_rows, page_name="master")

In [12]:
#main_dataframe = sup.negative_to_postive_entry_check(main_dataframe)

In [13]:
main_dataframe = pd.read_excel(r'resources\main_dataframe.xlsx')
main_dataframe1 = main_dataframe.copy()

In [14]:
main_dataframe1 = sup.rev_coloumn_data(main_dataframe1)

D:\Python Projects\doctors_payout\Scripts\doctors_payout\payout_app\excel_maker.py:404: RuntimeWarning: invalid value encountered in cast
  main_dataframe["DOCTOR_SHARE"].values.astype(int) > 1, "REV_STREAM"


In [15]:
main_dataframe1

,SERVICE_DATE,PATIENT_ID,EPISODE_ID,PATIENT_NAME,SERVICE_CODE,SERVICE_DESC,PATIENT_TYPE,GROSS_AMOUNT,NET_AMOUNT,DOCTOR_SHARE,DOCTOR_NAME,PHYSICIAN_ID,SOURCE OF DATA,COMMENTS,DOCTORS_GROUP,REFERENCE_GROUP,REV_STREAM
0,19/06/2023,KH1000854002,14114990,Mrs. Sulochna Rawat,CNIP000025,Doctor Routine Visit - Neurosurgery,Twin,2500.0,2500.0,2500.0,DR. AKSHAT KAYAL,51008200,IP Admitting Doc wise rev rep,NaN,D,IP Consultation,1.0
1,01/06/2023,KH1000207883,14132431,Mrs. Sajida Ahmed,CNIP000025,Doctor Routine Visit - Neurosurgery,General,2000.0,2000.0,2000.0,DR. AKSHAT KAYAL,51008200,IP Admitting Doc wise rev rep,NaN,D,IP Consultation,1.0
2,02/06/2023,KH1000207883,14132431,Mrs. Sajida Ahmed,CNIP000025,Doctor Routine Visit - Neurosurgery,General,2000.0,2000.0,2000.0,DR. AKSHAT KAYAL,51008200,IP Admitting Doc wise rev rep,NaN,D,IP Consultation,1.0
3,03/06/2023,KH1000207883,14132431,Mrs. Sajida Ahmed,CNIP000025,Doctor Routine Visit - Neurosurgery,General,2000.0,2000.0,2000.0,DR. AKSHAT KAYAL,51008200,IP Admitting Doc wise rev rep,NaN,D,IP Consultation,1.0
4,04/06/2023,KH1000207883,14132431,Mrs. Sajida Ahmed,CNIP000025,Doctor Routine Visit - Neurosurgery,General,3000.0,3000.0,3000.0,DR. AKSHAT KAYAL,51008200,IP Admitting Doc wise rev rep,NaN,D,IP Consultation,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45029,2023-06-29 08:26:24,KH1000885975,143075270001,NaN,CARD,NaN,EHC,400.0,400.0,400.0,Dr.PRAVIN KAHALE,NaN,EHC Query,NaN,C,OP Consultation,1.0
45030,2023-06-29 13:12:45,KH1000886167,143086050001,NaN,CARD,NaN,EHC,400.0,400.0,400.0,Dr.PRAVIN KAHALE,NaN,EHC Query,NaN,C,OP Consultation,1.0
45031,2023-06-29 13:13:28,KH1000886169,143086080001,NaN,CARD,NaN,EHC,400.0,400.0,400.0,Dr.PRAVIN KAHALE,NaN,EHC Query,NaN,C,OP Consultation,1.0
45032,2023-06-29 13:13:54,KH1000886172,143086200001,NaN,CARD,NaN,EHC,400.0,400.0,400.0,Dr.PRAVIN KAHALE,NaN,EHC Query,NaN,C,OP Consultation,1.0


In [16]:
sup.excel_generator(excel_data=main_dataframe1, page_name="main_dataframe1")

'D:\\Python Projects\\doctors_payout\\Scripts\\doctors_payout\\payout_app/excel_files/main_dataframe1-27-Jul-2023-12-55-08-378494.xlsx'